In [1]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, '/FileStore/tables/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

# Save and load model
model.save(sc, "target/tmp/myRandomForestClassificationModel2")
sameModel = RandomForestModel.load(sc, "target/tmp/myRandomForestClassificationModel")



Test Error = 0.0
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

 Tree 0:
 If (feature 406 <= 9.5)
 Predict: 0.0
 Else (feature 406 > 9.5)
 If (feature 327 <= 19.5)
 Predict: 1.0
 Else (feature 327 > 19.5)
 Predict: 0.0
 Tree 1:
 If (feature 371 <= 9.0)
 If (feature 606 <= 41.5)
 If (feature 356 <= 19.0)
 Predict: 1.0
 Else (feature 356 > 19.0)
 Predict: 0.0
 Else (feature 606 > 41.5)
 Predict: 0.0
 Else (feature 371 > 9.0)
 If (feature 343 <= 252.5)
 Predict: 0.0
 Else (feature 343 > 252.5)
 Predict: 1.0
 Tree 2:
 If (feature 461 <= 46.5)
 If (feature 379 <= 47.5)
 Predict: 0.0
 Else (feature 379 > 47.5)
 If (feature 575 <= 141.5)
 Predict: 1.0
 Else (feature 575 > 141.5)
 Predict: 0.0
 Else (feature 461 > 46.5)
 Predict: 1.0